## Tasks Guardrails


Task guardrails provide a way to validate and transform task outputs before they are passed to the next task. This feature helps ensure data quality and provides feedback to agents when their output doesn’t meet specific criteria

## CrewAI supports two types of guardrails:

## 1. Function-based guardrails:

 Python functions with custom validation logic, giving you complete control over the validation process and ensuring reliable, deterministic results.


## 2. LLM-based guardrails:

 String descriptions that use the agent’s LLM to validate outputs based on natural language criteria. These are ideal for complex or subjective validation requirements.



it is in tuple format -> (Bool,Any)

In [7]:
from crewai import Agent, LLM,TaskOutput
from pydantic import BaseModel
from typing import Tuple,Any
from dotenv import load_dotenv
import os
from crewai import Crew
load_dotenv() 


llm = LLM(model="groq/llama-3.3-70b-versatile",
        verbose = True,
        temperature = 0.2,
        api_key = os.getenv("GROQ_API_KEY")
        )


In [10]:
def validate_summary_length(task_output: TaskOutput)->Tuple[bool,Any]:
    try:
        print("Validating summary length")
        task_str_output = str(task_output)
        total_words = len(task_str_output.split())

        print(f"Word count: {total_words}")

        if total_words > 200:
            print("Summary exceeds 150 words")
            return (False, f"Summary exceeds 150 words. Word count: {total_words}")

        if total_words == 0:
            print("Summary is empty")
            return (False, "Generated summary is empty.")

        print("Summary is valid")
        return (True, task_output)

    except Exception as e:
        print("Validation system error")
        return (False, f"Validation system error: {str(e)}")


In [11]:
from crewai import Task, Agent

blog_agent = Agent(
    role="Professional Blog Writer",
    goal="Write an engaging, informative blog post strictly in 200 words based on the user-provided topic.",
    backstory=(
        "You are an expert content writer specializing in high-quality, SEO-friendly blog posts. "
        "You write clear, engaging, and structured content tailored to the given topic. "
        "You strictly follow word count requirements and never exceed or fall short of the specified limit."
    ),
    verbose=True,
    llm=llm
)

blog_task = Task(
    description=(
        "Write a well-structured blog post strictly in 200 words on the following topic:\n\n"
        "Topic: {topic}\n\n"
        "Instructions:\n"
        "- The blog must be exactly 200 words.\n"
        "- Use a clear introduction, body, and conclusion.\n"
        "- Keep the content engaging and informative.\n"
        "- Do not include headings or extra explanations.\n"
        "- Output only the blog content.\n"
    ),
    expected_output="A strictly 200-word blog post based on the provided topic.",
    agent=blog_agent,
    guardrail=validate_summary_length,  # validates exactly 200 words
    max_retries=3
)

In [12]:
crew = Crew(
    agents=[blog_agent],
    tasks=[blog_task],
    verbose=True
)

In [ ]:
result = crew.kickoff(inputs={"topic":"AI in Automation"})

In [ ]:
print(result)

In [16]:
from pydantic import BaseModel

class ResearchReport(BaseModel):
    """Represents a structured research report"""
    title: str
    summary: str
    key_findings: list[str]

In [13]:
import json
from typing import Tuple, Any

def validate_json_report(result: TaskOutput) -> Tuple[bool, Any]:
    """Ensures AI-generated output is valid JSON with required fields."""
    try:
        # Parse JSON output
        data = json.loads(result.pydantic.model_dump_json())

        # Check required fields
        if "title" not in data or "summary" not in data or "key_findings" not in data:
            return (False, "Missing required fields: title, summary, or key_findings.")

        return (True, result)  # JSON is valid
    except json.JSONDecodeError:
        return (False, "Invalid JSON format. Please ensure correct syntax.")


In [14]:
from crewai import Agent

# Create the AI Agent
research_report_agent = Agent(
    role="Research Analyst",
    goal="Generate structured JSON reports for research papers",
    backstory="You are an expert in technical writing and structured reporting.",
    verbose=False,
    llm=llm
)

In [17]:
from crewai import Task

research_report_task = Task(
    description="Generate a structured research report in valid JSON format.",
    expected_output="A valid JSON object containing 'title', 'summary', and 'key_findings'.",
    agent=research_report_agent,
    output_pydantic=ResearchReport,  # Ensures structured output
    guardrail=validate_json_report,  # Validate output before passing to next step
    max_retries=3  # Allow up to 3 retries if validation fails
)

In [18]:
from crewai import Crew

research_crew = Crew(
    agents=[research_report_agent],
    tasks=[research_report_task],
    verbose=True  # Display execution details
)

In [ ]:
result = research_crew.kickoff()

# Display the validated JSON output
print("Final Research Report:", result.pydantic)

## LLM-Based Guardrails (String Descriptions)

Instead of writing custom validation functions, you can use string descriptions that leverage LLM-based validation. When you provide a string to the guardrail or guardrails parameter, CrewAI automatically creates an LLMGuardrail that uses the agent’s LLM to validate the output based on your description.
Requirements:
The task must have an agent assigned (the guardrail uses the agent’s LLM)
Provide a clear, descriptive string explaining the validation criteria

## Requirements:
The task must have an agent assigned (the guardrail uses the agent’s LLM)
Provide a clear, descriptive string explaining the validation criteria


In [ ]:

blog_agent = Agent(
    role = "Technology Blog Writer",
    goal = "Create concise, engaging blog posts about technology topics in under 200 words",
    backstory = "You are an experienced technology writer with a talent for making complex topics "
    "accessible and engaging. You specialize in creating clear, well-structured content that "
    "educates readers without overwhelming them. Your writing style is conversational yet informative, "
    "and you excel at distilling key insights into digestible formats.",
    llm = llm,
    verbose = True
)

# Single LLM-based guardrail
blog_task = Task(
    description="Write an engaging blog post about the impact of artificial intelligence in education. "
    "Focus on practical applications, benefits for students and teachers, and emerging trends. "
    "Structure the content with clear sections using bullet points where appropriate to enhance readability.",
    expected_output="A well-structured blog post under 200 words that is accessible to general readers, "
    "uses bullet points for key information, avoids technical jargon, and maintains an engaging, "
    "conversational tone throughout.",
    agent=blog_agent,
    guardrail="The blog post must be under 200 words, written in plain language without technical jargon, "
    "and be easily understandable by readers with no technical background"
)

In [ ]:
crew = Crew(
    agents=[blog_agent],
    tasks=[blog_task],
    verbose= True,
)

In [ ]:
print(crew.kickoff())

## Multiple Guardrails

You can apply multiple guardrails to a task using the guardrails parameter. Multiple guardrails are executed sequentially, with each guardrail receiving the output from the previous one. This allows you to chain validation and transformation steps.

## The guardrails parameter accepts:

A list of guardrail functions or string descriptions
A single guardrail function or string (same as guardrail)
Note: If guardrails is provided, it takes precedence over guardrail. The guardrail parameter will be ignored when guardrails is set.

In [19]:
from typing import Tuple, Any
from crewai import TaskOutput, Task

def validate_word_count(result: TaskOutput) -> Tuple[bool, Any]:
    """Validate word count is within limits."""
    word_count = len(result.raw.split())
    if word_count < 100:
        return (False, f"Content too short: {word_count} words. Need at least 100 words.")
    if word_count > 500:
        return (False, f"Content too long: {word_count} words. Maximum is 500 words.")
    return (True, result.raw)

def validate_no_profanity(result: TaskOutput) -> Tuple[bool, Any]:
    """Check for inappropriate language."""
    profanity_words = ["badword1", "badword2"]  # Example list
    content_lower = result.raw.lower()
    for word in profanity_words:
        if word in content_lower:
            return (False, f"Inappropriate language detected: {word}")
    return (True, result.raw)

def format_output(result: TaskOutput) -> Tuple[bool, Any]:
    """Format and clean the output."""
    formatted = result.raw.strip()
    # Capitalize first letter
    formatted = formatted[0].upper() + formatted[1:] if formatted else formatted
    return (True, formatted)

# Apply multiple guardrails sequentially
blog_task = Task(
    description="Write a blog post about AI",
    expected_output="A well-formatted blog post between 100-500 words",
    agent=blog_agent,
    guardrails=[
        validate_word_count,      # First: validate length
        validate_no_profanity,    # Second: check content
        format_output             # Third: format the result
    ],
    guardrail_max_retries=3
)

In this example, the guardrails execute in order:

validate_word_count checks the word count

validate_no_profanity checks for inappropriate language (using the output from step 1)

format_output formats the final result (using the output from step 2)

If any guardrail fails, the error is sent back to the agent, and the task is retried up to 
guardrail_max_retries times.

Mixing function-based and LLM-based guardrails:

You can combine both function-based and string-based guardrails in the same list:

## Mixing function-based and LLM-based guardrails:

You can combine both function-based and string-based guardrails in the same list:


In [20]:
from typing import Tuple, Any
from crewai import TaskOutput, Task

def validate_word_count(result: TaskOutput) -> Tuple[bool, Any]:
    """Validate word count is within limits."""
    word_count = len(result.raw.split())
    if word_count < 100:
        return (False, f"Content too short: {word_count} words. Need at least 100 words.")
    if word_count > 500:
        return (False, f"Content too long: {word_count} words. Maximum is 500 words.")
    return (True, result.raw)

# Mix function-based and LLM-based guardrails
blog_task = Task(
    description="Write a blog post about AI",
    expected_output="A well-formatted blog post between 100-500 words",
    agent=blog_agent,
    guardrails=[
        validate_word_count,  # Function-based: precise word count check
        "The content must be engaging and suitable for a general audience",  # LLM-based: subjective quality check
        "The writing style should be clear, concise, and free of technical jargon"  # LLM-based: style validation
    ],
    guardrail_max_retries=3
)

## Guardrail Function Requirements

Function Signature:

Must accept exactly one parameter (the task output)

Should return a tuple of (bool, Any)

Type hints are recommended but optional

Return Values:

On success: it returns a tuple of (bool, Any). For example: (True, validated_result)

On Failure: it returns a tuple of (bool, str). For example: (False, "Error message explain the failure")

In [ ]:
'''andling Guardrail Results
When a guardrail returns (False, error):
The error is sent back to the agent
The agent attempts to fix the issue
The process repeats until:
The guardrail returns (True, result)
Maximum retries are reached (guardrail_max_retries)
Example with retry handling:'''